In [2]:
import cv2 as cv
import os
import face_recognition
from face_recognition import face_encodings
from face_recognition import load_image_file
from face_recognition import face_locations
from face_recognition import compare_faces
import dlib

#### Training model

In [3]:
known_face_encoded_images = []
know_face_names = []

In [4]:
train_path = os.listdir('./People')

for img_id, each_image in enumerate(train_path):
    if each_image.split('.')[1] == 'png':
        image = load_image_file('./People/'+each_image)
        known_face_encoded_images.append(face_encodings(image)[0])
        know_face_names.append(each_image.split('.')[0])

In [5]:
know_face_names

['Sanjit', 'Subhadeep', 'Subhamay']

In [6]:
known_face_encoded_images

[array([-0.09816214,  0.08969104,  0.0120691 , -0.02310877,  0.05490562,
        -0.05791179, -0.03882476, -0.09277894,  0.20006908, -0.04667822,
         0.22129439, -0.00559432, -0.12355392, -0.08977634,  0.02214807,
         0.05669685, -0.129418  , -0.13843566, -0.01281623, -0.05321051,
        -0.0097071 ,  0.05131897, -0.01742673,  0.07038455, -0.16149038,
        -0.33292922, -0.05501276, -0.14865157,  0.02248425, -0.02139379,
         0.01673915,  0.06060378, -0.1602093 , -0.01915496,  0.02863917,
         0.14861378, -0.01561518,  0.03904484,  0.16705817,  0.02428349,
        -0.12153091, -0.00139563,  0.04491469,  0.28361633,  0.1879925 ,
         0.05489397,  0.02917845, -0.03257489,  0.04518488, -0.22407955,
         0.07289708,  0.09854867,  0.11960498,  0.08118364,  0.10118857,
        -0.14710891, -0.00842232,  0.04076591, -0.17677446,  0.12445249,
         0.01919145,  0.0012072 , -0.02950026,  0.06694581,  0.30930758,
         0.12424819, -0.13626289, -0.08589881,  0.1

#### Testing model

In [10]:
classifier = cv.CascadeClassifier('./../8. Face Detection/haarcascade_frontalface_default.xml')

In [13]:
cam = cv.VideoCapture(0)

while True:
    _,img = cam.read()
    img = cv.flip(img,1)

    # extract live face and encoding it
    face = face_locations(img)
    face_encode = face_encodings(img, face)
    
    # compare it with other known faces
    for (top, right, bottom, left), encoded_face in zip(face, face_encode):
        matches = compare_faces(known_face_encoded_images, encoded_face)
        name = 'Unknown'

        if True in matches:
            first_match_index = matches.index(True)
            name = know_face_names[first_match_index]

    # display the name
    face_border = classifier.detectMultiScale(img, 1.1, 5)

    for x,y,w,h in face_border:
        cv.rectangle(img,(x,y),(x+w, y+h), (0,180,0), 2)
        cv.putText(img, name, (x,y-30),cv.FONT_HERSHEY_COMPLEX, 1, (0,180,0), 2)

    # capture image
    if cv.waitKey(10) == 13:
        cv.imwrite("Result.jpg",img)
    
    # display the web cam video
    cv.imshow("Frame",img)

    if cv.waitKey(10)==27:
        cam.release()
        cv.destroyAllWindows()
        break